<a href="https://colab.research.google.com/github/ekaterina533/dataset/blob/main/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Новый раздел

In [20]:
!pip install scapy dpkt pandas numpy scikit-learn matplotlib ipywidgets

In [3]:
%%writefile security_system.py
import os
import zipfile
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.ensemble import IsolationForest
import dpkt
import socket
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from typing import List, Dict, Any, Optional

# 1. Класс для анализа трафика
class TrafficAnalyzer:
    def __init__(self):
        self.model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
        self.encoders = {}
        self.scaler = RobustScaler()
        self.attack_stats = defaultdict(int)
        self.traffic_stats = defaultdict(int)
        self.feature_order = ['length', 'ttl', 'src_port', 'dst_port', 'src_ip', 'dst_ip', 'protocol']  # Фиксированный порядок

    def preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        """Предобработка данных с сохранением порядка признаков"""
        # Заполнение отсутствующих значений
        for col in ['src_port', 'dst_port']:
            if col in df.columns:
                df[col] = df[col].fillna(0).astype(int)

        # Кодирование категориальных признаков
        for col in ['src_ip', 'dst_ip', 'protocol']:
            if col in df.columns:
                self.encoders[col] = LabelEncoder()
                df[col] = self.encoders[col].fit_transform(df[col].astype(str))

        # Нормализация числовых признаков
        numeric_cols = ['length', 'ttl', 'src_port', 'dst_port']
        numeric_cols = [col for col in numeric_cols if col in df.columns]

        if numeric_cols:
            df[numeric_cols] = self.scaler.fit_transform(df[numeric_cols])

        # Возвращаем только нужные признаки в фиксированном порядке
        return df[[col for col in self.feature_order if col in df.columns]]

    def train(self, normal_traffic: pd.DataFrame):
        """Обучение модели с контролем порядка признаков"""
        # Оставляем только нужные столбцы в правильном порядке
        train_data = normal_traffic[[col for col in self.feature_order if col in normal_traffic.columns]]
        self.model.fit(train_data)

    def detect_anomalies(self, packets: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Обнаружение аномальных пакетов"""
        features = []
        valid_packets = []

        for pkt in packets:
            feat = self._extract_features(pkt)
            if feat is not None:
                features.append(feat)
                valid_packets.append(pkt)

        if features:
            try:
                # Создаем DataFrame с правильным порядком признаков
                feature_df = pd.DataFrame(features, columns=[col for col in self.feature_order if col in self.encoders or col in ['length', 'ttl', 'src_port', 'dst_port']])
                preds = self.model.predict(feature_df)
                return [pkt for pkt, pred in zip(valid_packets, preds) if pred == -1]
            except Exception as e:
                print(f"Ошибка предсказания: {str(e)}")
                print(f"Использованные признаки: {feature_df.columns.tolist()}")
                print(f"Ожидаемые признаки: {self.model.feature_names_in_}")
                return []
        return []

    def _extract_features(self, pkt: Dict[str, Any]) -> Optional[List[float]]:
        """Извлечение признаков из пакета"""
        try:
            features = []
            for col in self.feature_order:
                if col in ['length', 'ttl', 'src_port', 'dst_port']:
                    features.append(pkt.get(col, 0))
                elif col in self.encoders:
                    val = str(pkt.get(col, '0'))
                    if val in self.encoders[col].classes_:
                        features.append(self.encoders[col].transform([val])[0])
                    else:
                        features.append(0)
            return features
        except Exception as e:
            print(f"Ошибка извлечения признаков: {str(e)}")
            return None

# 2. Класс для управления трафиком
class TrafficManager:
    def __init__(self):
        self.blocked_ips = set()
        self.rate_limited_ips = {}
        self.load_history = []

    def get_recommendations(self, attack_type, src_ip):
        recommendations = []

        if attack_type == "DDoS":
            recommendations.extend([
                "1. Блокировать IP на firewall",
                "2. Ограничить скорость для этого IP",
                "3. Перенаправить трафик через scrubbing center"
            ])
        elif attack_type == "Port Scan":
            recommendations.extend([
                "1. Закрыть неиспользуемые порты",
                "2. Включить stealth mode",
                "3. Добавить IP в blacklist"
            ])

        recommendations.append("4. Отправить уведомление администратору")
        return recommendations

    def get_load_balance_recommendations(self, cpu_load, mem_load):
        recommendations = []

        if cpu_load > 80:
            recommendations.append("1. Увеличить количество worker-процессов")
            recommendations.append("2. Перенести часть нагрузки на backup-сервер")

        if mem_load > 80:
            recommendations.append("3. Оптимизировать кэширование")
            recommendations.append("4. Увеличить swap-пространство")

        if not recommendations:
            recommendations.append("Система работает в нормальном режиме")

        return recommendations

# 3. Парсер PCAP-файлов
def parse_pcap(file_path: str) -> List[Dict[str, Any]]:
    """Парсинг PCAP-файла в список пакетов"""
    packets = []
    with open(file_path, 'rb') as f:
        try:
            pcap = dpkt.pcap.Reader(f)
            for ts, buf in pcap:
                try:
                    eth = dpkt.ethernet.Ethernet(buf)
                    if not isinstance(eth.data, dpkt.ip.IP):
                        continue

                    ip = eth.data
                    transport = ip.data

                    packet = {
                        'timestamp': ts,
                        'src_ip': socket.inet_ntoa(ip.src),
                        'dst_ip': socket.inet_ntoa(ip.dst),
                        'length': ip.len,
                        'ttl': ip.ttl,
                        'protocol': 'other'
                    }

                    if isinstance(transport, dpkt.tcp.TCP):
                        packet.update({
                            'protocol': 'tcp',
                            'src_port': transport.sport,
                            'dst_port': transport.dport,
                            'flags': transport.flags
                        })
                    elif isinstance(transport, dpkt.udp.UDP):
                        packet.update({
                            'protocol': 'udp',
                            'src_port': transport.sport,
                            'dst_port': transport.dport
                        })

                    packets.append(packet)
                except Exception as e:
                    continue
        except Exception as e:
            print(f"Ошибка чтения файла {file_path}: {str(e)}")

    return packets

# 4. Интерфейс администратора
class AdminInterface:
    def __init__(self, analyzer, manager):
        self.analyzer = analyzer
        self.manager = manager
        self.setup_ui()

    def setup_ui(self):
        self.output = widgets.Output()
        self.recommendation_dropdown = widgets.Dropdown(options=[], description='Действие:')
        self.execute_button = widgets.Button(description="Выполнить")
        self.execute_button.on_click(self.execute_action)

        display(self.output)

    def show_attack_alert(self, attack_packets):
        with self.output:
            clear_output()
            print("🚨 Обнаружена потенциальная атака!")
            print(f"Тип атаки: {self.detect_attack_type(attack_packets)}")
            print("Рекомендуемые действия:")

            src_ip = attack_packets[0]['src_ip']
            recommendations = self.manager.get_recommendations(self.detect_attack_type(attack_packets), src_ip)
            self.recommendation_dropdown.options = recommendations

            display(self.recommendation_dropdown)
            display(self.execute_button)

    def show_load_alert(self, cpu_load, mem_load):
        with self.output:
            clear_output()
            print("⚖️ Нагрузка на систему:")
            print(f"CPU: {cpu_load}%, MEM: {mem_load}%")
            print("Рекомендации по балансировке:")

            recommendations = self.manager.get_load_balance_recommendations(cpu_load, mem_load)
            self.recommendation_dropdown.options = recommendations

            display(self.recommendation_dropdown)
            display(self.execute_button)

    def detect_attack_type(self, packets):
        if len(packets) > 1000:
            return "DDoS"
        elif len(set(p['dst_port'] for p in packets)) > 20:
            return "Port Scan"
        else:
            return "Unknown"

    def execute_action(self, b):
        with self.output:
            action = self.recommendation_dropdown.value
            print(f"Выполняется: {action}...")
            time.sleep(1)
            print("✅ Действие выполнено")

# 5. Главная система
class SecuritySystem:
    def __init__(self):
        self.analyzer = TrafficAnalyzer()
        self.manager = TrafficManager()
        self.interface = AdminInterface(self.analyzer, self.manager)

        # Имитация системного мониторинга
        self.cpu_load = 30
        self.mem_load = 45

    def load_dataset(self, pcap_files: List[str]) -> List[Dict[str, Any]]:
        all_packets = []
        for file in pcap_files:
            print(f"Обработка файла: {os.path.basename(file)}")
            packets = parse_pcap(file)
            print(f"Извлечено пакетов: {len(packets)}")

            # Проверка наличия обязательных полей
            if packets and all(key in packets[0] for key in ['length', 'ttl', 'src_ip', 'dst_ip', 'protocol']):
                all_packets.extend(packets)
            else:
                print(f"Файл {file} не содержит необходимых полей")

        if not all_packets:
            raise ValueError("Не удалось извлечь пакеты с необходимыми полями")

        df = pd.DataFrame(all_packets)
        processed_df = self.analyzer.preprocess(df)

        # Используем 70% данных для обучения
        train_size = int(0.7 * len(processed_df))
        self.analyzer.train(processed_df.iloc[:train_size])

        return all_packets

    def simulate_attacks(self, packets: List[Dict[str, Any]]):
        """Имитация обнаружения атак и высокой нагрузки"""
        # Проверяем первые 500 пакетов (увеличили для лучшего обнаружения)
        test_packets = packets[:1000]

        # Обнаружение аномалий
        attack_packets = self.analyzer.detect_anomalies(test_packets)
        if attack_packets:
            attack_type = self.interface.detect_attack_type(attack_packets)
            self.analyzer.attack_stats[attack_type] += len(attack_packets)
            print(f"Обнаружено {len(attack_packets)} аномальных пакетов ({attack_type})")
            self.interface.show_attack_alert(attack_packets)
        else:
            print("Аномалий не обнаружено")

        # Имитация высокой нагрузки (30% chance)
        if np.random.random() > 0.7:
            self.cpu_load = min(100, self.cpu_load + np.random.randint(20, 60))
            self.mem_load = min(100, self.mem_load + np.random.randint(10, 40))
            print(f"Нагрузка на систему: CPU={self.cpu_load}%, MEM={self.mem_load}%")
            self.interface.show_load_alert(self.cpu_load, self.mem_load)

# 6. Загрузка данных
def download_and_extract_zip():
    zip_url = "https://github.com/westermo/network-traffic-dataset/raw/main/data/extended/pcaps/right.zip"
    zip_path = "right.zip"
    extract_dir = "right"

    if not os.path.exists(extract_dir):
        print("⬇️ Загрузка ZIP-архива...")
        urlretrieve(zip_url, zip_path)

        print("📦 Распаковка архива...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

        os.remove(zip_path)

    return [os.path.join(extract_dir, f) for f in os.listdir(extract_dir) if f.endswith('.pcap')]

# 7. Запуск системы
def main():
    print("🔄 Запуск системы безопасности...")

    # Загрузка данных
    pcap_files = download_and_extract_zip()
    print(f"Найдено {len(pcap_files)} PCAP-файлов")

    # Инициализация системы
    system = SecuritySystem()
    packets = system.load_dataset(pcap_files)

    # Анализ трафика
    print("🔍 Анализ трафика...")
    system.simulate_attacks(packets)

    print("\n✅ Система готова к работе")
    print("Мониторинг трафика и нагрузки выполняется в фоновом режиме...")

if __name__ == "__main__":
    main()


Writing security_system.py


In [23]:
%%writefile generate_test_traffic.py
import random
import time
from scapy.all import *
from scapy.layers.inet import IP, TCP, UDP, Ether

def generate_normal_traffic(output_file="normal_traffic.pcap", packet_count=10000):
    """Генерация нормального трафика с Ethernet-заголовками"""
    packets = []
    for i in range(packet_count):
        eth = Ether(src="00:11:22:33:44:55", dst="66:77:88:99:aa:bb")
        ip = IP(src=f"192.168.{random.randint(0, 255)}.{random.randint(1, 254)}",
                dst=f"10.0.{random.randint(0, 255)}.{random.randint(1, 254)}")

        if random.choice([True, False]):
            pkt = eth/ip/TCP(sport=random.randint(1024, 65535),
                            dport=random.choice([80, 443, 22, 21]))
        else:
            pkt = eth/ip/UDP(sport=random.randint(1024, 65535),
                            dport=random.choice([53, 67, 68]))

        packets.append(pkt)

    wrpcap(output_file, packets)
    print(f"Сгенерировано {packet_count} нормальных пакетов в {output_file}")

def generate_ddos_attack(output_file="ddos_attack.pcap", packet_count=300):
    """Генерация DDoS атаки"""
    packets = []
    target_ip = "10.0.0.1"

    for i in range(packet_count):
        eth = Ether(src="00:11:22:33:44:55", dst="66:77:88:99:aa:bb")
        ip = IP(src=f"{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}",
                dst=target_ip)
        pkt = eth/ip/TCP(dport=80, flags="S")
        packets.append(pkt)

    wrpcap(output_file, packets)
    print(f"Сгенерировано {packet_count} DDoS пакетов в {output_file}")

if __name__ == "__main__":
    generate_normal_traffic()
    generate_ddos_attack()

Overwriting generate_test_traffic.py


In [5]:
%%writefile test_model.py
import os
from security_system import SecuritySystem

def test_model():
    system = SecuritySystem()
    test_files = ["normal_traffic.pcap", "ddos_attack.pcap"]

    for file in test_files:
        if not os.path.exists(file):
            print(f"Файл {file} не найден!")
            return

    print("\n=== Тест нормального трафика ===")
    normal_packets = system.load_dataset(["normal_traffic.pcap"])
    system.simulate_attacks(normal_packets)

    print("\n=== Тест DDoS атаки ===")
    attack_packets = system.load_dataset(["ddos_attack.pcap"])
    system.simulate_attacks(attack_packets)

if __name__ == "__main__":
    test_model()

Writing test_model.py


In [6]:
%%writefile perfomance_test.py
import time
import random
import pandas as pd
from security_system import TrafficAnalyzer

def test_performance():
    analyzer = TrafficAnalyzer()

    # Генерация тестовых данных
    data = {
        'src_ip': [f"192.168.1.{i}" for i in range(1, 1001)],
        'dst_ip': [f"10.0.0.{i%100}" for i in range(1, 1001)],
        'protocol': ['tcp' if i%2 else 'udp' for i in range(1, 1001)],
        'length': [random.randint(40, 1500) for _ in range(1000)],
        'ttl': [random.randint(32, 128) for _ in range(1000)],
        'src_port': [random.randint(1024, 65535) for _ in range(1000)],
        'dst_port': [random.choice([80, 443, 22, 53]) for _ in range(1000)]
    }
    df = pd.DataFrame(data)

    # Тест предобработки
    start_time = time.time()
    processed_df = analyzer.preprocess(df)
    preprocess_time = time.time() - start_time

    # Тест обучения
    start_time = time.time()
    analyzer.train(processed_df)
    train_time = time.time() - start_time

    # Тест предсказания
    test_data = df.iloc[:100].to_dict('records')
    start_time = time.time()
    anomalies = analyzer.detect_anomalies(test_data)
    predict_time = time.time() - start_time

    print("\n=== Результаты тестирования производительности ===")
    print(f"Время предобработки (1000 записей): {preprocess_time:.4f} сек")
    print(f"Время обучения модели: {train_time:.4f} сек")
    print(f"Время предсказания (100 записей): {predict_time:.4f} сек")
    print(f"Обнаружено аномалий: {len(anomalies)}")

if __name__ == "__main__":
    test_performance()

Writing perfomance_test.py


In [7]:
%%writefile accuracy_test.py
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from security_system import TrafficAnalyzer

def test_accuracy():
    # 1. Инициализация анализатора
    analyzer = TrafficAnalyzer()

    # 2. Генерация данных с фиксированным порядком признаков
    num_samples = 2000
    normal_samples = int(num_samples * 0.7)

    # Сначала создаем DataFrame с явным порядком столбцов
    columns_order = ['length', 'ttl', 'src_port', 'dst_port', 'src_ip', 'dst_ip', 'protocol']

    # Нормальный трафик
    normal_data = pd.DataFrame({
        'length': np.clip(np.random.normal(500, 100, normal_samples), 40, 1500).astype(int),
        'ttl': np.random.randint(50, 64, normal_samples),
        'src_port': np.random.randint(1024, 65535, normal_samples),
        'dst_port': np.random.choice([80, 443, 22, 53], normal_samples),
        'src_ip': [f"192.168.1.{i}" for i in range(normal_samples)],
        'dst_ip': [f"10.0.0.{i%10}" for i in range(normal_samples)],
        'protocol': ['tcp' if i%2 else 'udp' for i in range(normal_samples)]
    })[columns_order]  # Явно задаем порядок столбцов

    # Аномальный трафик
    attack_data = pd.DataFrame({
        'length': np.clip(np.random.normal(1500, 10, num_samples-normal_samples), 40, 1500).astype(int),
        'ttl': np.random.randint(32, 40, num_samples-normal_samples),
        'src_port': np.random.randint(1024, 65535, num_samples-normal_samples),
        'dst_port': np.random.randint(1, 1024, num_samples-normal_samples),
        'src_ip': [f"172.16.1.{i}" for i in range(num_samples-normal_samples)],
        'dst_ip': ["10.0.0.1"] * (num_samples-normal_samples),
        'protocol': ['tcp'] * (num_samples-normal_samples)
    })[columns_order]  # Тот же порядок столбцов

    # Объединение данных
    df = pd.concat([normal_data, attack_data], ignore_index=True)
    labels = np.array([1]*normal_samples + [-1]*(num_samples-normal_samples))

    # 3. Предобработка
    processed_df = analyzer.preprocess(df)

    # 4. Проверка порядка признаков после предобработки
    print("Порядок признаков после предобработки:", processed_df.columns.tolist())

    # 5. Разделение на train/test
    train_size = int(0.7 * len(processed_df))
    X_train = processed_df.iloc[:train_size]
    X_test = processed_df.iloc[train_size:]
    y_train = labels[:train_size]
    y_test = labels[train_size:]

    # 6. Обучение модели
    analyzer.train(X_train)

    # 7. Проверка feature_names_in_ в модели
    if hasattr(analyzer.model, 'feature_names_in_'):
        print("Признаки модели:", analyzer.model.feature_names_in_)

    # 8. Предсказание с явным контролем порядка признаков
    try:
        preds = analyzer.model.predict(X_test[analyzer.model.feature_names_in_])
        print("\n=== Отчет о точности модели ===")
        print(classification_report(y_test, preds,
                                 target_names=['Аномалия', 'Нормальный']))
    except Exception as e:
        print("\n=== Ошибка предсказания ===")
        print(str(e))
        print("\nСравнение признаков:")
        print("Ожидалось:", analyzer.model.feature_names_in_)
        print("Фактически:", X_test.columns.tolist())

if __name__ == "__main__":
    test_accuracy()

Writing accuracy_test.py


In [8]:
%%writefile integration_test.py
import os
from security_system import SecuritySystem

def run_integration_test():
    print("=== Запуск интеграционного теста системы безопасности ===")

    # 1. Инициализация системы
    system = SecuritySystem()
    print("[✓] Система инициализирована")

    # 2. Загрузка тестовых данных
    test_files = ["normal_traffic.pcap", "ddos_attack.pcap"]
    missing_files = [f for f in test_files if not os.path.exists(f)]

    if missing_files:
        print(f"[×] Отсутствуют тестовые файлы: {missing_files}")
        print("Сначала запустите generate_test_traffic.py")
        return

    try:
        packets = system.load_dataset(test_files)
        print("[✓] Тестовые данные успешно загружены")
    except Exception as e:
        print(f"[×] Ошибка загрузки данных: {str(e)}")
        return

    # 3. Тестирование обнаружения атак
    print("\nТестирование обнаружения атак...")
    system.simulate_attacks(packets)

    # 4. Проверка статистики
    print("\n=== Статистика обнаружения ===")
    for attack_type, count in system.analyzer.attack_stats.items():
        print(f"{attack_type}: {count} пакетов")

    print("\n=== Интеграционный тест завершен ===")

if __name__ == "__main__":
    run_integration_test()

Writing integration_test.py


In [24]:
!python generate_test_traffic.py

Сгенерировано 10000 нормальных пакетов в normal_traffic.pcap
Сгенерировано 300 DDoS пакетов в ddos_attack.pcap


In [10]:
!python test_model.py

Output()

=== Тест нормального трафика ===
Обработка файла: normal_traffic.pcap
Извлечено пакетов: 1000
Обнаружено 888 аномальных пакетов (Unknown)
🚨 Обнаружена потенциальная атака!
Тип атаки: Unknown
Рекомендуемые действия:
Dropdown(description='Действие:', options=('4. Отправить уведомление администратору',), value='4. Отправить уведомление администратору')
Button(description='Выполнить', style=ButtonStyle())

=== Тест DDoS атаки ===
Обработка файла: ddos_attack.pcap
Извлечено пакетов: 300
Обнаружено 22 аномальных пакетов (Unknown)
🚨 Обнаружена потенциальная атака!
Тип атаки: Unknown
Рекомендуемые действия:
Dropdown(description='Действие:', options=('4. Отправить уведомление администратору',), value='4. Отправить уведомление администратору')
Button(description='Выполнить', style=ButtonStyle())


In [11]:
!python perfomance_test.py


=== Результаты тестирования производительности ===
Время предобработки (1000 записей): 0.0098 сек
Время обучения модели: 0.1943 сек
Время предсказания (100 записей): 0.0129 сек
Обнаружено аномалий: 63


In [13]:
!python integration_test.py

=== Запуск интеграционного теста системы безопасности ===
Output()
[✓] Система инициализирована
Обработка файла: normal_traffic.pcap
Извлечено пакетов: 1000
Обработка файла: ddos_attack.pcap
Извлечено пакетов: 300
[✓] Тестовые данные успешно загружены

Тестирование обнаружения атак...
Обнаружено 792 аномальных пакетов (Unknown)
🚨 Обнаружена потенциальная атака!
Тип атаки: Unknown
Рекомендуемые действия:
Dropdown(description='Действие:', options=('4. Отправить уведомление администратору',), value='4. Отправить уведомление администратору')
Button(description='Выполнить', style=ButtonStyle())
Нагрузка на систему: CPU=81%, MEM=78%
⚖️ Нагрузка на систему:
CPU: 81%, MEM: 78%
Рекомендации по балансировке:
Dropdown(description='Действие:', options=('1. Увеличить количество worker-процессов', '2. Перенести часть нагрузки на backup-сервер'), value='1. Увеличить количество worker-процессов')
Button(description='Выполнить', style=ButtonStyle())

=== Статистика обнаружения ===
Unknown: 792 пакетов


In [14]:
!python security_system.py

🔄 Запуск системы безопасности...
⬇️ Загрузка ZIP-архива...
📦 Распаковка архива...
Найдено 1 PCAP-файлов
Output()
Обработка файла: right.pcap
Извлечено пакетов: 22902
🔍 Анализ трафика...
Обнаружено 927 аномальных пакетов (Port Scan)
🚨 Обнаружена потенциальная атака!
Тип атаки: Port Scan
Рекомендуемые действия:
Dropdown(description='Действие:', options=('1. Закрыть неиспользуемые порты', '2. Включить stealth mode', '3. Добавить IP в blacklist', '4. Отправить уведомление администратору'), value='1. Закрыть неиспользуемые порты')
Button(description='Выполнить', style=ButtonStyle())
Нагрузка на систему: CPU=76%, MEM=84%
⚖️ Нагрузка на систему:
CPU: 76%, MEM: 84%
Рекомендации по балансировке:
Dropdown(description='Действие:', options=('3. Оптимизировать кэширование', '4. Увеличить swap-пространство'), value='3. Оптимизировать кэширование')
Button(description='Выполнить', style=ButtonStyle())

✅ Система готова к работе
Мониторинг трафика и нагрузки выполняется в фоновом режиме...


In [27]:
%%writefile generate_test_traffic1.py
import random
import socket
import time
from scapy.all import *
from scapy.layers.inet import IP, TCP, UDP, ICMP

def generate_normal_traffic(output_file="normal_traffic.pcap", packet_count=1000):
    """Генерация нормального трафика"""
    packets = []
    for i in range(packet_count):
        # Генерация случайных IP-адресов в приватном диапазоне
        src_ip = f"192.168.{random.randint(0, 255)}.{random.randint(1, 254)}"
        dst_ip = f"10.0.{random.randint(0, 255)}.{random.randint(1, 254)}"

        # Случайный выбор протокола
        if random.choice([True, False]):
            pkt = IP(src=src_ip, dst=dst_ip)/TCP(sport=random.randint(1024, 65535),
                                              dport=random.choice([80, 443, 22, 21]))
        else:
            pkt = IP(src=src_ip, dst=dst_ip)/UDP(sport=random.randint(1024, 65535),
                                              dport=random.choice([53, 67, 68]))

        packets.append(pkt)

    wrpcap(output_file, packets)
    print(f"Сгенерировано {packet_count} нормальных пакетов в {output_file}")

def generate_ddos_attack(output_file="ddos_attack.pcap", packet_count=5000):
    """Генерация DDoS атаки"""
    packets = []
    target_ip = "10.0.0.1"  # Цель атаки

    for i in range(packet_count):
        src_ip = f"{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}"
        pkt = IP(src=src_ip, dst=target_ip)/TCP(dport=80, flags="S")
        packets.append(pkt)

    wrpcap(output_file, packets)
    print(f"Сгенерировано {packet_count} DDoS пакетов в {output_file}")

def generate_port_scan(output_file="port_scan.pcap", target_ip="10.0.0.1"):
    """Генерация Port Scan атаки"""
    packets = []
    src_ip = f"192.168.{random.randint(0, 255)}.{random.randint(1, 254)}"

    for port in range(1, 100):  # Сканируем первые 100 портов
        pkt = IP(src=src_ip, dst=target_ip)/TCP(dport=port, flags="S")
        packets.append(pkt)

    wrpcap(output_file, packets)
    print(f"Сгенерировано Port Scan пакетов в {output_file}")

Writing generate_test_traffic1.py


In [28]:
!python generate_test_traffic1.py

In [29]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler, LabelEncoder
from collections import defaultdict, Counter
from typing import List, Dict, Any, Optional
import logging
import dpkt  # Библиотека для работы с PCAP-файлами
import socket

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TrafficAnalyzer:
    def __init__(self):
        """Инициализация с оптимизированными параметрами модели"""
        self.model = IsolationForest(
            n_estimators=200,
            max_samples=256,
            contamination=0.01,
            random_state=42,
            verbose=1
        )
        self.encoders = {}
        self.scaler = RobustScaler()
        self.feature_order = [
            'length', 'ttl', 'src_port', 'dst_port',
            'src_ip', 'dst_ip', 'protocol'
        ]
        self.threshold = -0.2  # Начальное значение порога
        self.attack_stats = defaultdict(int)

    def preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        """Улучшенная предобработка с обработкой выбросов"""
        # Заполнение пропусков
        for col in ['src_port', 'dst_port', 'ttl']:
            if col in df.columns:
                df[col] = df[col].fillna(0).astype(int)

        # Обработка выбросов
        if 'length' in df.columns:
            df['length'] = np.clip(df['length'], 40, 1500)

        # Кодирование категориальных признаков
        for col in ['src_ip', 'dst_ip', 'protocol']:
            if col in df.columns:
                if col not in self.encoders:
                    self.encoders[col] = LabelEncoder()
                    df[col] = self.encoders[col].fit_transform(df[col].astype(str))

        # Нормализация числовых признаков
        numeric_cols = ['length', 'ttl', 'src_port', 'dst_port']
        numeric_cols = [col for col in numeric_cols if col in df.columns]

        if numeric_cols:
            df[numeric_cols] = self.scaler.fit_transform(df[numeric_cols])

        return df[[col for col in self.feature_order if col in df.columns]]

    def calibrate_model(self, normal_packets: List[Dict[str, Any]]):
        """Калибровка модели на чистом трафике"""
        logger.info("Начало калибровки модели...")
        df = pd.DataFrame(normal_packets)
        processed_df = self.preprocess(df)

        # Автоматическая настройка параметров
        self.model.fit(processed_df)
        scores = self.model.decision_function(processed_df)

        # Автоматический подбор порога (5% квантиль)
        self.threshold = np.percentile(scores, 5)
        logger.info(f"Установлен порог аномальности: {self.threshold:.2f}")

    def train(self, normal_traffic: pd.DataFrame):
        """Обучение модели с контролем качества"""
        logger.info("Обучение модели...")
        train_data = normal_traffic[self.feature_order]
        self.model.fit(train_data)
        logger.info("Обучение завершено")

    def detect_attack_type(self, packets: List[Dict[str, Any]]) -> str:
        """Точное определение типа атаки"""
        if not packets:
            return "Normal"

        # Анализ распределения пакетов
        src_ips = Counter(p['src_ip'] for p in packets)
        ports = Counter(p.get('dst_port', 0) for p in packets)

        if len(packets) > 500:
            if len(src_ips) > 50 and max(src_ips.values()) < 20:
                return "DDoS"
            return "Flood Attack"

        if len(ports) > 20:
            return "Port Scan"

        if any(p.get('length', 0) > 1400 for p in packets):
            return "Oversized Packets"

        if len(src_ips) == 1:
            return "Targeted Attack"

        return "Suspicious Activity"

    def detect_anomalies(self, packets: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Улучшенное обнаружение аномалий"""
        if not packets:
            return []

        features = []
        valid_packets = []

        for pkt in packets:
            feat = self._extract_features(pkt)
            if feat and all(v is not None for v in feat):
                features.append(feat)
                valid_packets.append(pkt)

        if not features:
            return []

        try:
            feature_df = pd.DataFrame(features, columns=self.feature_order)
            scores = self.model.decision_function(feature_df)

            # Фильтрация по порогу
            anomalies = [pkt for pkt, score in zip(valid_packets, scores) if score < self.threshold]
            logger.info(f"Обнаружено {len(anomalies)} аномальных пакетов")
            return anomalies
        except Exception as e:
            logger.error(f"Ошибка предсказания: {str(e)}")
            return []

    def _extract_features(self, pkt: Dict[str, Any]) -> Optional[List[float]]:
        """Извлечение признаков с обработкой ошибок"""
        try:
            features = []
            for col in self.feature_order:
                if col in ['length', 'ttl', 'src_port', 'dst_port']:
                    features.append(pkt.get(col, 0))
                elif col in self.encoders:
                    val = str(pkt.get(col, '0'))
                    if val in self.encoders[col].classes_:
                        features.append(self.encoders[col].transform([val])[0])
                    else:
                        features.append(0)
                else:
                    features.append(0)
            return features
        except Exception as e:
            logger.error(f"Ошибка извлечения признаков: {str(e)}")
            return None

class SecuritySystem:
    def __init__(self):
        self.analyzer = TrafficAnalyzer()
        logger.info("Система безопасности инициализирована")

    @staticmethod
    def parse_pcap(file_path: str) -> List[Dict[str, Any]]:
        """Парсинг PCAP-файла с обработкой TCP/UDP/ICMP пакетов"""
        packets = []

        try:
            with open(file_path, 'rb') as f:
                pcap = dpkt.pcap.Reader(f)

                for ts, buf in pcap:
                    packet = {
                        'timestamp': ts,
                        'length': len(buf)
                    }

                    try:
                        eth = dpkt.ethernet.Ethernet(buf)
                        if not isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
                            continue

                        ip = eth.data
                        packet.update({
                            'src_ip': socket.inet_ntoa(ip.src) if hasattr(ip, 'src') else '0.0.0.0',
                            'dst_ip': socket.inet_ntoa(ip.dst) if hasattr(ip, 'dst') else '0.0.0.0',
                            'ttl': ip.ttl if hasattr(ip, 'ttl') else 0,
                            'protocol': ip.p if hasattr(ip, 'p') else 0
                        })

                        # Обработка TCP
                        if isinstance(ip.data, dpkt.tcp.TCP):
                            tcp = ip.data
                            packet.update({
                                'src_port': tcp.sport,
                                'dst_port': tcp.dport
                            })
                        # Обработка UDP
                        elif isinstance(ip.data, dpkt.udp.UDP):
                            udp = ip.data
                            packet.update({
                                'src_port': udp.sport,
                                'dst_port': udp.dport
                            })
                        # Обработка ICMP
                        elif isinstance(ip.data, dpkt.icmp.ICMP):
                            packet.update({
                                'src_port': 0,
                                'dst_port': 0
                            })

                        packets.append(packet)
                    except Exception as e:
                        logger.warning(f"Ошибка парсинга пакета: {str(e)}")
                        continue

        except FileNotFoundError:
            logger.error(f"Файл не найден: {file_path}")
        except Exception as e:
            logger.error(f"Ошибка чтения PCAP-файла: {str(e)}")

        return packets

    def load_dataset(self, pcap_files: List[str]) -> List[Dict[str, Any]]:
        """Загрузка и обработка данных"""
        all_packets = []
        for file in pcap_files:
            logger.info(f"Обработка файла: {file}")
            packets = self.parse_pcap(file)
            logger.info(f"Извлечено пакетов: {len(packets)}")

            if packets and all(key in packets[0] for key in ['length', 'src_ip']):
                all_packets.extend(packets)
            else:
                logger.warning(f"Файл {file} содержит неполные данные")

        if not all_packets:
            raise ValueError("Не удалось загрузить данные")

        df = pd.DataFrame(all_packets)
        processed_df = self.analyzer.preprocess(df)

        # Калибровка на первых 70% данных
        train_size = int(0.7 * len(processed_df))
        self.analyzer.calibrate_model(processed_df.iloc[:train_size])
        self.analyzer.train(processed_df.iloc[:train_size])

        return all_packets

    def analyze_traffic(self, packets: List[Dict[str, Any]]):
        """Анализ трафика с выводом результатов"""
        anomalies = self.analyzer.detect_anomalies(packets)
        if anomalies:
            attack_type = self.analyzer.detect_attack_type(anomalies)
            self.analyzer.attack_stats[attack_type] += len(anomalies)

            logger.warning(f"Обнаружена атака: {attack_type}")
            logger.warning(f"Количество пакетов: {len(anomalies)}")
            logger.warning(f"Источник: {anomalies[0]['src_ip']}")

            return {
                "attack_type": attack_type,
                "count": len(anomalies),
                "samples": anomalies[:5]  # Первые 5 аномальных пакетов
            }
        return {"status": "Normal traffic"}

def main():
    """Точка входа в систему"""
    try:
        logger.info("Запуск системы безопасности...")
        system = SecuritySystem()

        # Тестовые данные (замените на реальные файлы)
        pcap_files = ["normal_traffic.pcap", "ddos_attack.pcap"]
        packets = system.load_dataset(pcap_files)

        # Анализ трафика
        result = system.analyze_traffic(packets)
        logger.info(f"Результат анализа: {result}")

    except Exception as e:
        logger.error(f"Критическая ошибка: {str(e)}")
    finally:
        logger.info("Работа системы завершена")

if __name__ == "__main__":
    main()

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
